# Classification de Toxicite avec BERT

Ce notebook implemente un modele de classification multi-label utilisant BERT (Bidirectional Encoder Representations from Transformers).

## Labels de toxicite:
- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

## 1. Installation et Imports

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, hamming_loss
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Device (GPU si disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Device: cpu


## 2. Chargement des Donnees

In [6]:
# Charger le dataset
df = pd.read_csv('train.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColonnes: {df.columns.tolist()}")
df.head()

Dataset shape: (159571, 8)

Colonnes: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
# Labels
LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print("Distribution des labels:")
print(df[LABEL_COLS].sum())
print(f"\nTotal: {len(df)} commentaires")
print(f"Toxiques: {(df[LABEL_COLS].sum(axis=1) > 0).sum()}")
print(f"Non-toxiques: {(df[LABEL_COLS].sum(axis=1) == 0).sum()}")

Distribution des labels:
toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

Total: 159571 commentaires
Toxiques: 16225
Non-toxiques: 143346


## 3. Echantillonnage (pour accelerer l'entrainement)

BERT est couteux en calcul. On utilise un echantillon pour le developpement.

In [8]:
# Echantillonner les donnees (ajuster selon votre GPU)
# Pour un entrainement complet, mettre SAMPLE_SIZE = None
SAMPLE_SIZE = 10000  # Reduire si memoire insuffisante

if SAMPLE_SIZE:
    # Echantillonnage stratifie
    df_toxic = df[df[LABEL_COLS].sum(axis=1) > 0]
    df_clean = df[df[LABEL_COLS].sum(axis=1) == 0]
    
    n_toxic = min(len(df_toxic), SAMPLE_SIZE // 2)
    n_clean = min(len(df_clean), SAMPLE_SIZE // 2)
    
    df_sample = pd.concat([
        df_toxic.sample(n=n_toxic, random_state=RANDOM_STATE),
        df_clean.sample(n=n_clean, random_state=RANDOM_STATE)
    ]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
    
    print(f"Echantillon: {len(df_sample)} commentaires")
    print(f"Toxiques: {(df_sample[LABEL_COLS].sum(axis=1) > 0).sum()}")
    print(f"Non-toxiques: {(df_sample[LABEL_COLS].sum(axis=1) == 0).sum()}")
else:
    df_sample = df
    print(f"Dataset complet: {len(df_sample)} commentaires")

Echantillon: 10000 commentaires
Toxiques: 5000
Non-toxiques: 5000


## 4. Preparation des Donnees pour BERT

In [9]:
# Hyperparametres
MAX_LENGTH = 128  # Longueur max des sequences (reduire si memoire insuffisante)
BATCH_SIZE = 16   # Reduire si memoire insuffisante
EPOCHS = 3
LEARNING_RATE = 2e-5

print(f"MAX_LENGTH: {MAX_LENGTH}")
print(f"BATCH_SIZE: {BATCH_SIZE}")
print(f"EPOCHS: {EPOCHS}")
print(f"LEARNING_RATE: {LEARNING_RATE}")

MAX_LENGTH: 128
BATCH_SIZE: 16
EPOCHS: 3
LEARNING_RATE: 2e-05


In [10]:
# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(f"Tokenizer charge: bert-base-uncased")
print(f"Vocabulaire: {tokenizer.vocab_size} tokens")

Tokenizer charge: bert-base-uncased
Vocabulaire: 30522 tokens


In [11]:
# Split train/validation/test
train_df, temp_df = train_test_split(
    df_sample, test_size=0.2, random_state=RANDOM_STATE,
    stratify=(df_sample[LABEL_COLS].sum(axis=1) > 0).astype(int)
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=RANDOM_STATE,
    stratify=(temp_df[LABEL_COLS].sum(axis=1) > 0).astype(int)
)

print(f"Train: {len(train_df)} samples")
print(f"Validation: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

Train: 8000 samples
Validation: 1000 samples
Test: 1000 samples


In [12]:
class ToxicDataset(Dataset):
    """
    Dataset PyTorch pour la classification de toxicite.
    """
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        labels = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [13]:
# Creer les datasets
train_dataset = ToxicDataset(
    texts=train_df['comment_text'].values,
    labels=train_df[LABEL_COLS].values,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)

val_dataset = ToxicDataset(
    texts=val_df['comment_text'].values,
    labels=val_df[LABEL_COLS].values,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)

test_dataset = ToxicDataset(
    texts=test_df['comment_text'].values,
    labels=test_df[LABEL_COLS].values,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Val dataset: {len(val_dataset)} samples")
print(f"Test dataset: {len(test_dataset)} samples")

Train dataset: 8000 samples
Val dataset: 1000 samples
Test dataset: 1000 samples


In [14]:
# Creer les DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

Train batches: 500
Val batches: 63
Test batches: 63


## 5. Definition du Modele BERT

In [15]:
class BertToxicClassifier(nn.Module):
    """
    Classificateur de toxicite base sur BERT.
    
    Architecture:
    - BERT base (pretraine)
    - Dropout
    - Couche lineaire pour classification multi-label
    """
    def __init__(self, n_classes=6, dropout=0.3):
        super(BertToxicClassifier, self).__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        # BERT output
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # Utiliser le [CLS] token
        pooled_output = outputs.pooler_output
        
        # Dropout + Classification
        output = self.dropout(pooled_output)
        output = self.classifier(output)
        
        return output

In [16]:
# Initialiser le modele
model = BertToxicClassifier(n_classes=len(LABEL_COLS))
model = model.to(device)

print(f"Modele charge sur {device}")
print(f"\nArchitecture:")
print(f"- BERT hidden size: {model.bert.config.hidden_size}")
print(f"- Dropout: 0.3")
print(f"- Output: {len(LABEL_COLS)} classes")

# Nombre de parametres
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nParametres totaux: {total_params:,}")
print(f"Parametres entrainables: {trainable_params:,}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 

## 6. Configuration de l'Entrainement

In [ ]:
# Loss function (BCE pour multi-label)
criterion = nn.BCEWithLogitsLoss()

# Optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

# Scheduler
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

print(f"Loss: BCEWithLogitsLoss")
print(f"Optimizer: AdamW (lr={LEARNING_RATE})")
print(f"Total steps: {total_steps}")
print(f"Warmup steps: {int(0.1 * total_steps)}")

## 7. Fonctions d'Entrainement et d'Evaluation

In [ ]:
def train_epoch(model, data_loader, criterion, optimizer, scheduler, device):
    """
    Entraine le modele pour une epoch.
    """
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    progress_bar = tqdm(data_loader, desc='Training')
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        
        # Predictions
        preds = torch.sigmoid(outputs).cpu().detach().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(data_loader)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Calcul F1 (seuil 0.5)
    preds_binary = (all_preds >= 0.5).astype(int)
    f1_micro = f1_score(all_labels, preds_binary, average='micro')
    f1_macro = f1_score(all_labels, preds_binary, average='macro')
    
    return avg_loss, f1_micro, f1_macro

In [ ]:
def evaluate(model, data_loader, criterion, device):
    """
    Evalue le modele sur un dataset.
    """
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    preds_binary = (all_preds >= 0.5).astype(int)
    f1_micro = f1_score(all_labels, preds_binary, average='micro')
    f1_macro = f1_score(all_labels, preds_binary, average='macro')
    
    return avg_loss, f1_micro, f1_macro, all_preds, all_labels

## 8. Entrainement du Modele

In [ ]:
# Entrainement
print("="*70)
print("ENTRAINEMENT DU MODELE BERT")
print("="*70)

best_val_f1 = 0
history = {'train_loss': [], 'val_loss': [], 'train_f1': [], 'val_f1': []}

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print("-"*40)
    
    # Train
    train_loss, train_f1_micro, train_f1_macro = train_epoch(
        model, train_loader, criterion, optimizer, scheduler, device
    )
    
    # Validation
    val_loss, val_f1_micro, val_f1_macro, _, _ = evaluate(
        model, val_loader, criterion, device
    )
    
    # Historique
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_f1'].append(train_f1_micro)
    history['val_f1'].append(val_f1_micro)
    
    print(f"\nTrain Loss: {train_loss:.4f} | Train F1 (micro): {train_f1_micro:.4f} | Train F1 (macro): {train_f1_macro:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val F1 (micro): {val_f1_micro:.4f} | Val F1 (macro): {val_f1_macro:.4f}")
    
    # Sauvegarder le meilleur modele
    if val_f1_micro > best_val_f1:
        best_val_f1 = val_f1_micro
        torch.save(model.state_dict(), 'bert_toxic_best.pt')
        print(f"[Meilleur modele sauvegarde] Val F1: {val_f1_micro:.4f}")

print("\n" + "="*70)
print("ENTRAINEMENT TERMINE")
print("="*70)
print(f"Meilleur Val F1: {best_val_f1:.4f}")

## 9. Visualisation de l'Entrainement

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(history['val_loss'], label='Val Loss', marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss par Epoch')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# F1 Score
axes[1].plot(history['train_f1'], label='Train F1', marker='o')
axes[1].plot(history['val_f1'], label='Val F1', marker='o')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('F1 Score (micro)')
axes[1].set_title('F1 Score par Epoch')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Evaluation sur le Test Set

In [ ]:
# Charger le meilleur modele
model.load_state_dict(torch.load('bert_toxic_best.pt'))
print("Meilleur modele charge.")

# Evaluation sur test set
test_loss, test_f1_micro, test_f1_macro, test_preds, test_labels = evaluate(
    model, test_loader, criterion, device
)

print("\n" + "="*70)
print("RESULTATS SUR LE TEST SET")
print("="*70)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test F1 (micro): {test_f1_micro:.4f}")
print(f"Test F1 (macro): {test_f1_macro:.4f}")

In [ ]:
# Predictions binaires
test_preds_binary = (test_preds >= 0.5).astype(int)

# Metriques detaillees
print("\nMetriques par label:")
print("-"*50)

for i, label in enumerate(LABEL_COLS):
    f1 = f1_score(test_labels[:, i], test_preds_binary[:, i])
    print(f"{label:<15}: F1 = {f1:.4f}")

# Hamming Loss
h_loss = hamming_loss(test_labels, test_preds_binary)
print(f"\nHamming Loss: {h_loss:.4f}")

In [ ]:
# Classification report pour chaque label
print("\nRapport de classification detaille:")
print("="*70)

for i, label in enumerate(LABEL_COLS):
    print(f"\n--- {label.upper()} ---")
    print(classification_report(
        test_labels[:, i], 
        test_preds_binary[:, i],
        target_names=['Non-toxic', 'Toxic'],
        zero_division=0
    ))

## 11. Test sur des Exemples

In [ ]:
def predict_text(text, model, tokenizer, device, threshold=0.5):
    """
    Predit la toxicite d'un texte.
    """
    model.eval()
    
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs).cpu().numpy()[0]
    
    predictions = {}
    for i, label in enumerate(LABEL_COLS):
        predictions[label] = {
            'probability': float(probs[i]),
            'prediction': int(probs[i] >= threshold)
        }
    
    return predictions

In [ ]:
# Tester sur des exemples
test_texts = [
    "You are stupid and I hate you!",
    "Thank you for your help, I really appreciate it.",
    "Go to hell you idiot!",
    "I disagree with your opinion but I respect it.",
    "This is absolutely ridiculous and you should be ashamed!"
]

print("="*70)
print("TEST SUR DES EXEMPLES")
print("="*70)

for text in test_texts:
    print(f"\nTexte: {text}")
    print("-"*50)
    
    preds = predict_text(text, model, tokenizer, device)
    
    toxic_labels = []
    for label, info in preds.items():
        if info['prediction'] == 1:
            toxic_labels.append(f"{label} ({info['probability']:.2%})")
    
    if toxic_labels:
        print(f"TOXIQUE: {', '.join(toxic_labels)}")
    else:
        print("NON TOXIQUE")

## 12. Sauvegarde du Modele Final

In [ ]:
import pickle

# Sauvegarder le modele complet
torch.save({
    'model_state_dict': model.state_dict(),
    'label_cols': LABEL_COLS,
    'max_length': MAX_LENGTH,
    'history': history,
    'test_metrics': {
        'f1_micro': test_f1_micro,
        'f1_macro': test_f1_macro,
        'hamming_loss': h_loss
    }
}, 'bert_toxic_classifier.pt')

print("Modele sauvegarde dans 'bert_toxic_classifier.pt'")
print("\nContenu:")
print("  - model_state_dict: poids du modele")
print("  - label_cols: liste des labels")
print("  - max_length: longueur max des sequences")
print("  - history: historique d'entrainement")
print("  - test_metrics: metriques sur le test set")

## 13. Comparaison avec les Autres Modeles

In [ ]:
# Comparaison des modeles (a remplir avec les resultats des autres modeles)
print("="*70)
print("COMPARAISON DES MODELES")
print("="*70)

print(f"\n{'Modele':<30} {'F1 Micro':<12} {'F1 Macro':<12}")
print("-"*55)
print(f"{'BERT':<30} {test_f1_micro:<12.4f} {test_f1_macro:<12.4f}")
print(f"{'XGBoost (toxic_classifier)':<30} {'---':<12} {'---':<12}")
print(f"{'Hybride NB/LR/RF':<30} {'---':<12} {'---':<12}")
print("\n(Remplir avec les resultats des autres modeles pour comparer)")

## 14. Prochaines Etapes

Pour ameliorer les performances:

1. **Augmenter les donnees**: Utiliser tout le dataset (pas d'echantillonnage)
2. **Fine-tuning**: Ajuster les hyperparametres (learning rate, epochs, batch size)
3. **Modeles plus grands**: Essayer bert-large-uncased ou RoBERTa
4. **Gestion du desequilibre**: Class weights ou focal loss
5. **Optimisation des seuils**: Trouver le seuil optimal par label
6. **Ensemble**: Combiner BERT avec les modeles classiques